### NICO

In [15]:
import numpy as np
from random import random, shuffle

Initialize parameters

In [16]:
n = 4 #number of nodes in the network
T = 100 #number of paths
Nm = 6 #number of nodes per path

In [17]:
A = np.random.rand(n, n)
A = A/A.sum(axis = 1, keepdims=True)

In [18]:
Pi = np.random.rand(n, 1)
Pi = Pi/Pi.sum(axis=0, keepdims=True)

Generate some paths according to this Markov model

In [19]:
X = np.zeros((T, Nm))

In [20]:
cumprobs = Pi.cumsum(axis = 0)

In [21]:
for walk in X:
    larger = (cumprobs >= random()).nonzero()
    walk[0] = larger[0][0]
    for i in range(n):
        if i < n: 
            cumprobs = A[i,:].cumsum(axis=0)
            larger = (cumprobs >= random()).nonzero()
            walk[i + 1] = larger[0][0]

Shuffle observations

In [22]:
Y = X.copy()
for walk in Y:
    shuffle(walk)

In [ ]:
numTrials = 50
